In [1]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

2025-07-05 06:33:34.222295: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751697214.454559      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751697214.519026      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Set paths to your extracted folder
base_path = ("/kaggle/input/binary-clasification")
train_dir = os.path.join("/kaggle/input/binary-clasification/train/train")
val_dir = os.path.join("/kaggle/input/binary-clasification/val/val")

In [3]:
# Image parameters
img_height, img_width = 150, 150
batch_size = 32


In [4]:
# Image data generators
train_datagen = ImageDataGenerator(rescale=1.0 / 255)
val_datagen = ImageDataGenerator(rescale=1.0 / 255)


In [5]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)


Found 1926 images belonging to 2 classes.


In [6]:
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

Found 422 images belonging to 2 classes.


In [7]:
# Model definition
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-07-05 06:33:51.614853: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [8]:
# Compile model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [9]:
# Train model
model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 45s 689ms/step - accuracy: 0.7959 - loss: 0.8662 - val_accuracy: 0.8128 - val_loss: 0.4863
Epoch 2/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 41s 672ms/step - accuracy: 0.8617 - loss: 0.3552 - val_accuracy: 0.8341 - val_loss: 0.3736
Epoch 3/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 39s 647ms/step - accuracy: 0.8782 - loss: 0.2706 - val_accuracy: 0.8649 - val_loss: 0.3378
Epoch 4/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 39s 645ms/step - accuracy: 0.9245 - loss: 0.1850 - val_accuracy: 0.8744 - val_loss: 0.3419
Epoch 5/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 40s 649ms/step - accuracy: 0.9571 - loss: 0.1333 - val_accuracy: 0.8697 - val_loss: 0.3436
Epoch 6/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 43s 680ms/step - accuracy: 0.9659 - loss: 0.1040 - val_accuracy: 0.8886 - val_loss: 0.3705
Epoch 7/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 40s 657ms/step - accuracy: 0.9696 - loss: 0.0879 - val_accuracy: 0.8815 - val_loss: 0.4160
Epoch 8/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 40s 659ms/step - accuracy: 0.9798 - loss: 0.0631 - val_accu

In [11]:
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
import numpy as np

# Get predictions on training data
train_generator.reset()
y_train_true = train_generator.classes
y_train_pred_prob = model.predict(train_generator, verbose=1)
y_train_pred = (y_train_pred_prob > 0.5).astype(int)

# Get predictions on validation data
val_generator.reset()
y_val_true = val_generator.classes
y_val_pred_prob = model.predict(val_generator, verbose=1)
y_val_pred = (y_val_pred_prob > 0.5).astype(int)

# Calculate and print metrics
print("---- Training Metrics ----")
print("Precision:", precision_score(y_train_true, y_train_pred))
print("Recall:   ", recall_score(y_train_true, y_train_pred))
print("F1 Score: ", f1_score(y_train_true, y_train_pred))

print("\n---- Validation Metrics ----")
print("Precision:", precision_score(y_val_true, y_val_pred))
print("Recall:   ", recall_score(y_val_true, y_val_pred))
print("F1 Score: ", f1_score(y_val_true, y_val_pred))


61/61 ━━━━━━━━━━━━━━━━━━━━ 13s 216ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 239ms/step
---- Training Metrics ----
Precision: 0.8348736906962415
Recall:    0.8348736906962415
F1 Score:  0.8348736906962415

---- Validation Metrics ----
Precision: 0.8067226890756303
Recall:    0.8396501457725948
F1 Score:  0.8228571428571428


In [12]:
print("\nClassification Report (Validation):")
print(classification_report(y_val_true, y_val_pred, target_names=val_generator.class_indices))



Classification Report (Validation):
              precision    recall  f1-score   support

      female       0.15      0.13      0.14        79
        male       0.81      0.84      0.82       343

    accuracy                           0.71       422
   macro avg       0.48      0.48      0.48       422
weighted avg       0.68      0.71      0.69       422



In [13]:
# Save model
model.save("gender_classifier_model.h5")